# Week 3 - Managing tools and depdendencies

## Why do we care?

Sometimes a project you are working on needs a specific version of node or python. 

And almost always the project you are working on needs its own versions of depdendencies (libraries that you install).

## Managing tool versions

There are several tools for managing tool versions:

- `nvm` node version manager
- `pyenv` python version manager
- `asdf` universal version manager for node, python, java, php, etc.
- `mise` like asdf but better in some ways (used to be called rtx)

I will describe mise.

## MISE: installation

    curl https://mise.jdx.dev/install.sh | sh

## MISE: activation

(1) Run the first line if you use the bash shell, the second if you use the zsh shell, or the third if you use the fish shell. By default WSL uses the bash shell and MacOS uses the zsh shell.

    echo 'eval "$(~/.local/bin/mise activate bash)"' >> ~/.bashrc
    
    echo 'eval "$(~/.local/bin/mise activate zsh)"' >> ~/.zshrc
    
    echo '~/.local/bin/mise activate fish | source' >> ~/.config/fish/config.fish

(2) Now open a new termainal and verify everything was successful:

    mise doctor

## MISE: installing tools

First, cd to the root directory of your project, then run

    mise use python@3.11
    
This will download python 3.11 and automatically make it the active version when you are in that directory. Then if you ran

    mise use python@3.9

in a different directory, python 3.9 would be the active version when you were in that directory.

## MISE: configuration

This file tells mise which tool versions should be active for this directory.

    cat .mise.toml

## Virtual Environments

You will install different depdendencies (using `npm` for node or `pip` for python) for different projects you work on. You don't want the dependencies for different projects to get mixed together!

In node, your dependencies are automatically placed in a `node_modules` subdirectory of your project. Node is smart enough to know that when you are in a project directory, you want to get the dependencies from the `node_modules` subdirectory.

Python doesn't do this automatically for you. You have to create the depdendency directory and tell python to use it.

## Creating virtual environments in python

In python, project dependencies by convention are stored in a `.venv` subdirectory of your project. `venv` stands for *virtual environment*. When the virtual environment is activated, then `pip` will put dependencies in the `.venv` directory and python will read them from the `.venv` directory, so they don't conflict with the dependencies in a different python project. 

To create the `.venv` subdirectory for your project's virtual environment, run this command in your project directory:

    python -m venv .venv

## Activating virtual environments in python

You must also remember to activate it each time work on the project, before running pip to install dependencies by typing the first line for bash or zsh, or the second line for fish:

    source .venv/bin/activate
    source .venv/bin/activate.fish

If you forget to activate it before installing dependencies with pip, then those dependencies will go into the global python environment. In general, you should avoid this. You don't want to use the global python environment.

To deactivate your python environment type `deactivate`

## Installing python dependencies using pip

Pip is a package manager for python, similar to how npm is a package manager for node.

To install a dependency (called requests) using pip:

    pip install requests

## Using your virtual environment in VS Code

Open VS Code on the current directory

    code .
    
Create a new python file called **test.py** that uses the requests dependency:

    import requests
    r = requests.get("https://jsonplaceholder.typicode.com/posts/1")
    print(r.json())
    
Before you can execute this file, you need to tell VS Code which python environment you want to run it in. Type **control-shift-p** to open the command palette, followed by **python: select interpreter**, and click on **('.venv': venv)**

## Sharing your python virtual environment with others

In node, dependency versions go in `package.json` and `package-lock.json` so others can install the same dependency versions that you have. 

Python doesn't do this automatically. You have to create the dependency versions file yourself. By convention this file is called `requirements.txt`

## Creating requirements.txt

An easy way to create requirements.txt is by running:

    pip freeze > requirements.txt

which writes every dependency in your current python virtual environment along with their versions to requirements.txt, very similar to package-lock.json. 

I don't like this approach, because it includes all of your dependencies' dependencies. It results in a huge requirements.txt file and it's impossible to see what your true top-level project dependencies are. It's as if you just had package-lock.json and not package.json.

## A better way to create requirements.txt

Another way to create requirements.txt is each time you use pip to install a dependency:

    pip install requests

which installs requests plus all of its dependencies, then use pip freeze followed by grep to find the version of requests that got installed and append that to your requirements.txt file:

    pip freeze | grep "requests==" >> requirements.txt

## Installing dependencies from requirements.txt

Once you have created the requirements.txt file, check it into git.

When someone else clones your project, they first create the virtual environment directory:

    python -m venv .venv

And activate the virtual environment by typing one of the following:

    source .venv/bin/activate
    source .venv/bin/activate.fish

Then install the dependencies listed in requirements.txt

    pip install -r requirements.txt`

## MISE to the rescue

It can be easy to forget to activate the virtual environment. Fortunately, mise can do that for you automatically:

Edit `.mise.toml` and change `python="3.11"` to 

    python = {version="3.11", virtualenv=".venv"}

Then run (this only needs to be done once):

    mise settings set experimental true
    mise settings set python_venv_auto_create true

MISE will now create and activate the python virtual environment automatically whenever you cd to your project directory.

## Conclusion

- MISE can do a lot more than this. You can use MISE to install node, and MISE can also set environment variables automatically when you enter a directory. Learn more at https://mise.jdx.dev/ 

- There is an even better way to manage virtual environments and dependencies than venv and pip, and it's called `poetry`. It's a bit more complicated than venv and pip so we won't learn about it right away, but we will learn about it in a few months. I use venv and pip for simple projects and `poetry` for larger projects.